# Big Data Challenge


## ETL process using RDS database

In [9]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.1.3'
# spark_version = 'spark-3.<version number>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:14 http://ppa.launchpad.net/c2d4u.tea

In [10]:
# Conn to Postgres
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar


--2022-08-15 12:13:09--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.1’

postgresql-42.2.9.j 100%[===================>] 892.61K  4.88MB/s    in 0.2s    

2022-08-15 12:13:10 (4.88 MB/s) - ‘postgresql-42.2.9.jar.1’ saved [914037/914037]



---
### **Extract**

* Use the provided schema to create tables in your RDS database.

* Create two separate Google Colab notebooks and extract any two datasets from the list at review dataset: https://s3.amazonaws.com/amazon-reviews-pds/tsv/index.txt. Put each dataset into its own notebook.
Note: You could ETL both data sources in a single Colab notebook, but it will be easier to work with these large S3 data sources in separate notebooks.

* Be sure to handle the header correctly. If you read the file without the header parameter, you may find that the column headers are included in the table rows.

In [11]:
# create spark session ------> AmazonCamerasReviews
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("AmazonCamerasReviews").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [12]:
# Read in data from my S3 Bucket
from pyspark import SparkFiles
# url from file in bucket 
url="https://diana-big-data-challenge.s3.amazonaws.com/amazon_reviews_us_Camera_v1_00.tsv"
# add file
spark.sparkContext.addFile(url)
# read
camera_reviews_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Camera_v1_00.tsv"), sep='\t', header=True, inferSchema=True)

# Show DataFrame
camera_reviews_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    2975964|R1NBG94582SJE2|B00I01JQJM|     860486164|GoPro Rechargeabl...|          Camera|          5|            0|          0|   N|                Y|          Five Stars|                  ok| 2015-08-31|
|         US|   23526356|R273DCA6Y0H9V7|B00TCO0ZAA|     292641483|Professional 58mm...|          Camera|          5|    

In [14]:
# Count the number of records (rows) in the dataset
df_rows = camera_reviews_df.count()
print(f'Number of rows: {df_rows}')

Number of rows: 1801974


---
### Transform

Transform the dataset to fit the tables in the schema file. Be sure that the DataFrames match in data type and in column name.

In [15]:
camera_reviews_df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [16]:
from pyspark.sql.types import * 
camera_reviews_df = camera_reviews_df.withColumn("review_date", camera_reviews_df["review_date"].cast(DateType()))

In [17]:
camera_reviews_df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: date (nullable = true)



```
CREATE TABLE review_id_table (
  review_id TEXT PRIMARY KEY NOT NULL,
  customer_id INTEGER,
  product_id TEXT,
  product_parent INTEGER,
  review_date DATE -- this should be in the formate yyyy-mm-dd
);
```



In [18]:
review_id_cols = camera_reviews_df.select(["review_id","customer_id","product_id","product_parent","review_date"])
review_id_cols.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R1NBG94582SJE2|    2975964|B00I01JQJM|     860486164| 2015-08-31|
|R273DCA6Y0H9V7|   23526356|B00TCO0ZAA|     292641483| 2015-08-31|
| RQVOXO7WUOFK6|   52764145|B00B7733E0|      75825744| 2015-08-31|
|R1KWKSF21PO6HO|   47348933|B006ZN4U34|     789352955| 2015-08-31|
|R38H3UO1J190GI|   33680700|B00HUEBGMU|      19067902| 2015-08-31|
|R3NPIFKLR19NQA|   30301059|B008MW6Y12|     597683407| 2015-08-31|
|R3MBE6UCH3435E|   28282645|B00TE8XKIS|      35563334| 2015-08-31|
|R2E7A4FF0PVY5Q|     502818|B00ZKDUFBQ|     555190742| 2015-08-31|
| R3R8JDQ2BF4NM|    1481233|B010BZ7S2Q|     129544315| 2015-08-31|
|R1YND4BS823GN5|   27885926|B00HRXSSRA|     708418657| 2015-08-31|
|R2TZNSA18V7YF6|    3183883|B005C95NM4|     246957815| 2015-08-31|
|R22ZVRDPPXIDNL|   23208852|B00LBIBYBE|     746593019| 2015-08

In [19]:
review_id_cols.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- review_date: date (nullable = true)



```
-- This table will contain only unique values
CREATE TABLE products (
  product_id TEXT PRIMARY KEY NOT NULL UNIQUE,
  product_title TEXT
);
```



In [20]:
products_cols = camera_reviews_df.select(["product_id","product_title"])
products_cols = products_cols.dropDuplicates()
products_cols.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00L9ZF5BY|Flylinktech Actio...|
|B00017LSF8|B+W Clear UV Haze...|
|B00CGTN2N0|Queen Spy Mini U8...|
|B002POMZWU|Usb cable for kod...|
|B00FEY2BCW|Camera Case Eveca...|
|B0131XNIDS|*5-pack* microSue...|
|B004OUMTT2|Celestron PowerSe...|
|B004HY3XOA|58MM UV Lens Filt...|
|B00LBFKLRU|(Create a generic...|
|B007QXZ6D0|Vello Cinch Strap...|
|B00PG36JTW|Tamrac TEK Fitted...|
|B003JNJDGM|TTL Off Camera Sh...|
|B00ZYATKCA|Olympus Air Mirro...|
|B00S2X48IW|Qkoo Selfie Handh...|
|B00VQJASPG|    Canon SL1 Bundle|
|B00554PCDG|Pixel Flash Hot S...|
|B00006JPWW|Panasonic AY-DVMC...|
|B00B9Z23NM|Nikon 8250 ACULON...|
|B00GOJ7JGY|Fotasy HET63 Lens...|
|B00IUU22WG|White Photography...|
+----------+--------------------+
only showing top 20 rows



In [21]:
products_cols.printSchema()

root
 |-- product_id: string (nullable = true)
 |-- product_title: string (nullable = true)





```
-- Customer table for first data set
CREATE TABLE customers (
  customer_id INT PRIMARY KEY NOT NULL UNIQUE,
  customer_count INT
);

```



In [22]:
customers_cols = camera_reviews_df.groupby("customer_id").agg({"customer_id": "count"})
customers_cols.show()

+-----------+------------------+
|customer_id|count(customer_id)|
+-----------+------------------+
|   52695798|                 1|
|   48363612|                 5|
|   46909180|                 1|
|   45595220|                 1|
|   50372387|                 2|
|    9731896|                 1|
|   24540309|                 1|
|    2019000|                 1|
|   50798385|                 5|
|   37669883|                 1|
|   19718301|                 1|
|   45616772|                 1|
|    2167730|                 1|
|   47027968|                 7|
|    5459822|                 1|
|   52484883|                 1|
|   12425248|                 1|
|   15460750|                 1|
|   46944960|                 1|
|   37502310|                 2|
+-----------+------------------+
only showing top 20 rows



In [23]:
customers_cols = customers_cols.withColumnRenamed("count(customer_id)", "customer_count")
customers_cols.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   52695798|             1|
|   48363612|             5|
|   46909180|             1|
|   45595220|             1|
|   50372387|             2|
|    9731896|             1|
|   24540309|             1|
|    2019000|             1|
|   50798385|             5|
|   37669883|             1|
|   19718301|             1|
|   45616772|             1|
|    2167730|             1|
|   47027968|             7|
|    5459822|             1|
|   52484883|             1|
|   12425248|             1|
|   15460750|             1|
|   46944960|             1|
|   37502310|             2|
+-----------+--------------+
only showing top 20 rows



In [24]:
customers_cols.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_count: long (nullable = false)



In [25]:
customers_cols = customers_cols.withColumn("customer_count", customers_cols["customer_count"].cast(IntegerType()))

In [26]:
customers_cols.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_count: integer (nullable = false)





```
-- vine table
CREATE TABLE vine_table (
  review_id TEXT PRIMARY KEY,
  star_rating INTEGER,
  helpful_votes INTEGER,
  total_votes INTEGER,
  vine TEXT
);
```



In [27]:
vine_cols = camera_reviews_df.select(["review_id","star_rating","helpful_votes","total_votes","vine"])
vine_cols.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R1NBG94582SJE2|          5|            0|          0|   N|
|R273DCA6Y0H9V7|          5|            0|          0|   N|
| RQVOXO7WUOFK6|          2|            1|          1|   N|
|R1KWKSF21PO6HO|          5|            0|          0|   N|
|R38H3UO1J190GI|          5|            1|          1|   N|
|R3NPIFKLR19NQA|          3|            0|          0|   N|
|R3MBE6UCH3435E|          3|            8|          8|   N|
|R2E7A4FF0PVY5Q|          5|            0|          1|   N|
| R3R8JDQ2BF4NM|          5|            0|          2|   N|
|R1YND4BS823GN5|          1|            0|          0|   N|
|R2TZNSA18V7YF6|          4|            1|          1|   N|
|R22ZVRDPPXIDNL|          5|            0|          0|   N|
|R1F4O6WOO2W461|          3|            3|          3|   N|
| RT1KLS3QSJNUT|          5|            

In [28]:
vine_cols.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)



---
### Load

Load the DataFrames that correspond to tables into an RDS instance. Note: This process can take up to 10 minutes for each. Ensure that everything is correct before uploading.

In [41]:
mode = "append"
jdbc_url="jdbc:postgresql://big-data-1.cffyyt6b3iba.us-east-1.rds.amazonaws.com:5432/cameras_db"
config = {"user":"root", 
          "password": "postgres",
          "driver":"org.postgresql.Driver"}

In [42]:
review_id_cols.write.jdbc(url=jdbc_url, table="review_id_table", mode=mode, properties=config)

In [43]:
products_cols.write.jdbc(url=jdbc_url, table="products", mode=mode, properties=config)

In [44]:
customers_cols.write.jdbc(url=jdbc_url, table="customers", mode=mode, properties=config)

In [45]:
vine_cols.write.jdbc(url=jdbc_url, table="vine_table", mode=mode, properties=config)

---